In [1]:
%load_ext autoreload
%autoreload 2
%load_ext sql 
%pylab inline
%matplotlib inline

import matplotlib.text as text

import scipy.interpolate as interp
import pandas as pd
#from mp.txpepa import *
from sklearn import (manifold, datasets, decomposition, ensemble, lda,
                     random_projection)
from sklearn.metrics.pairwise import euclidean_distances,manhattan_distances
import statsmodels.api as sm
import numpy.linalg as LA
import bio.hts.traj as traj
#from rpy2.robjects.packages import importr
#from rpy2.robjects.vectors import FloatVector
#stats = importr('stats')
from sklearn.neighbors import KNeighborsClassifier

DAT_DIR = '/share/home/ishah/projects/Chem/data/tables/'
PKL_DIR = '/share/home/ishah/projects/Chem/data/pickle/'
RES_DIR = '/share/home/ishah/projects/Chem/data/results/'
FIG_DIR = '/share/home/ishah/projects/Chem/figs/readacross/'

import pickle,time
import pylab as pl

tmstmp = time.strftime("%m-%d-%Y",time.localtime())
tmstmp

Populating the interactive namespace from numpy and matplotlib


/usr/local/anaconda2/lib/python2.7/site-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)


'09-22-2017'

In [2]:
from bio.data.genra import *


In [5]:
G3 = openMongo(user='ishah',passwd='ishah',db='genra_v3',host='pb.epa.gov')
DB = openMongo(user='ishah',passwd='ishah',db='ghelman',host='pb.epa.gov')

In [6]:
DB.collection_names()

[u'test_compounds',
 u'logs',
 u'test_comps',
 u'toxval',
 u'fingeprints',
 u'fingerprints',
 u'meta',
 u'test_tox',
 u'test_logs',
 u'compounds',
 u'test_meta']

In [7]:
DB.compounds.find_one().keys()

[u'predicted_props',
 u'smiles',
 u'dsstox_sid',
 u'chemspider_id',
 u'measured_props',
 u'created_at',
 u'iupac',
 u'updated_at',
 u'pubchem_cid',
 u'gsid',
 u'synonyms',
 u'dsstox_cid',
 u'mol_weight',
 u'inchi_key',
 u'_id',
 u'casrn',
 u'name']

In [10]:
DB.logs.find_one()

{u'_id': ObjectId('58fe6103f0e291b4c06a1929'),
 u'dsstox_cid': u'DTXCID101',
 u'logs': [u'Added to database',
  {u'collection': u'compounds',
   u'compound': u'DTXCID101',
   u'field': u'predicted_props',
   u'prop_name': u'ACD_BP',
   u'time': datetime.datetime(2017, 7, 7, 18, 24, 11),
   u'type': u'add',
   u'value': 440.899},
  {u'collection': u'compounds',
   u'compound': u'DTXCID101',
   u'field': u'predicted_props',
   u'prop_name': u'ACD_FP',
   u'time': datetime.datetime(2017, 7, 7, 18, 24, 11),
   u'type': u'add',
   u'value': 250.661},
  {u'collection': u'compounds',
   u'compound': u'DTXCID101',
   u'field': u'predicted_props',
   u'prop_name': u'ACD_Prop_Density',
   u'time': datetime.datetime(2017, 7, 26, 18, 24, 49),
   u'type': u'add',
   u'value': 1.393},
  {u'collection': u'compounds',
   u'compound': u'DTXCID101',
   u'field': u'predicted_props',
   u'prop_name': u'ACD_Prop_Index_Of_Refraction',
   u'time': datetime.datetime(2017, 7, 26, 18, 24, 49),
   u'type': u'add

In [12]:
DB.logs.distinct('logs.field')

[u'predicted_props']

In [42]:
DB.compounds.find_one({'name':{'$regex':'carbon tetrachloride','$options':'i'}})

{u'_id': ObjectId('58fe610df0e291b4c06a1bb5'),
 u'casrn': u'56-23-5',
 u'chemspider_id': 5730,
 u'created_at': datetime.datetime(2017, 4, 24, 20, 32, 19),
 u'dsstox_cid': u'DTXCID20250',
 u'dsstox_sid': u'DTXSID8020250',
 u'gsid': 20250,
 u'inchi_key': u'VZGDMQKNWNREIO-UHFFFAOYSA-N',
 u'iupac': u'Tetrachloromethane',
 u'measured_props': {u'Acute Oral LD50': [2349.74, 2860.0, 2350.0, 2350.0],
  u'Atmos- Hydroxylation Rate': [1.2e-16],
  u'Bioconcentration Factor': [7.4131],
  u'Biodegration Class': [0.0],
  u'Boiling Point': [77.0, 76.6667, 76.8],
  u'Domestic Manufacturing Production': [None,
   None,
   None,
   2480600.0,
   None,
   None,
   0.0,
   None,
   None],
  u'Fish Biotrans- Half-Life (Km)': [0.0645654],
  u"Henry's Law": [0.0276],
  u'Imported Volume': [0.0, None, 0.0, 0.0, 0.0, None, 763400.0, 0.0, None],
  u'LogKoa: Octanol-Air': [2.79],
  u'LogP: Octanol-Water': [2.83],
  u'Melting Point': [-22.7778, -23.0, -22.92, -23.0, -23.0],
  u'PubMed_counts': [11059.0],
  u'Soil 

In [14]:
DB.toxval.find_one()

[u'_id', u'dsstox_cid', u'toxvals']

In [49]:
DB.toxval.find_one(dict(dsstox_cid='DTXCID20250'))

{u'_id': ObjectId('58fe610df0e291b4c06a1bb6'),
 u'dsstox_cid': u'DTXCID20250',
 u'toxvals': [{u'casrn': u'56-23-5',
   u'chemical_id': 38,
   u'details_text': u'IARC details',
   u'details_url': u'http://ag.epa.gov:8528/actorws/toxval/v01/toxval_source?source=iarc&casrn=56-23-5',
   u'dsstox_compound_id': u'DTXCID20250',
   u'dsstox_substance_id': u'DTXSID8020250',
   u'efk_dsstox_source_substance_id': 4978514,
   u'exposure_method': u'diet',
   u'exposure_method_original': u'-',
   u'exposure_route': u'-',
   u'exposure_route_original': u'-',
   u'grouping_id': 5410,
   u'human_eco': u'human health',
   u'media': u'-',
   u'media_original': u'-',
   u'name': u'Carbon tetrachloride',
   u'sex': u'-',
   u'sex_original': u'-',
   u'source': u'IARC',
   u'source_source_id': u'-',
   u'source_url': u'https://rais.ornl.gov/tools/tox_profiles.html',
   u'species': u'-',
   u'species_original': u'-',
   u'strain': u'-',
   u'strain_original': u'-',
   u'study_duration_class': u'-',
   u'stud

In [44]:
Tox=pd.DataFrame(DB.toxval.find_one(dict(dsstox_cid='DTXCID30182'))['toxvals'])

In [38]:
#Tox.head(10).T

In [46]:
#print "\n".join(Tox.columns)

In [47]:
(Tox.groupby(['source','study_type','study_duration_class','species','strain','sex','toxval_type','toxval_subtype','toxval_text'])
 .aggregate(dict(toxval_numeric=[np.median,np.std],
                 #toxval_numeric_qualifier=lambda x: '.'.join(set(x)),
                 toxval_units = lambda x: '.'.join(set(x)),
                 #source_url = lambda x: '.'.join(set(x)),
                )
           )
)

toxval_numeric  \
                                                                                                                                                                                                                median   
source   study_type                   study_duration_class           species strain              sex toxval_type                        toxval_subtype                         toxval_text                               
ACToR    -                            -                              -       -                   -   MEG                                -                                      -                             57.500000   
                                                                                                     RfD                                inhalation                             -                              0.000000   
                                                                                                                                        oral                                   -                              0.050000   
                                                                                                     aquatic toxicity scale             -                                      -                            100.000000   
                                                                                                     cancer classification              -                                      -                              0.500000   
                                                                                                     cancer slope factor                oral                                   -                              0.000000   
                                                                                                     chronic toxicity score             -                                      -                           1000.000000   
                                                                                                     cumulative daily concentration     -                                      -                              3.700000   
                                                                                                     cumulative daily intake            estimated                              -                              0.000185   
                                                                                                     mammalian toxicity score           -                                      -                           5000.000000   
                                                                                                     modifying factor                   RfD                                    -                              1.000000   
                                                                                                     occupational exposure limit        8 hour                                 -                             10.000000   
                                                                                                     protective action criteria         PAC-2, irreversable or serious effects -                              5.000000   
                                                                                                                                        PAC-3, life-threatening effects        -                            240.000000   
                                                                                                     screening level                    groundwater                            -                             77.000000   
                                                                                                                                        soil                                   -                            310.000000   
                                                          

In [48]:
G3.collection_names()

[u'chm_fp',
 u'tox2_fp',
 u'tox_fp_0',
 u'tox_fp',
 u'bio_fp_info',
 u'genra_perf_v1',
 u'toxref',
 u'genra_perf_v2',
 u'tox5_fp',
 u'chm_alerts',
 u'tx21_fp',
 u'compound',
 u'rattox_fp',
 u'bio_fp',
 u'fp_stats',
 u'tox4_fp',
 u'bio_fp_0',
 u'assay_info',
 u'genra_perf_v1_sum_v1',
 u'clusters1']